In [265]:
import pandas as pd 
import numpy as np
import time
from timeit import default_timer as timer
from datetime import timedelta

# The Task

- Calculate the value of the ETF using the ETF formula 
- Isolate all the unique ISINs into a separate dataframe
- Increment each ticker by 1 + a normally distributed return 
- Assimilate all the unique ISINs back into the respective ETF dataframes 
- Recalculate their values 

## Loading each of the ETFs into a Pandas DataFrame
- The number of shares outstanding for each ETF was taken from the iShares website directly and stored as an integer 

In [415]:
IS15 = pd.read_csv("IS15_holdings.csv").dropna()
IS15.columns = IS15.iloc[0]
IS15 = IS15.iloc[1: , :]
IS15_so = 12100283

ISXF = pd.read_csv("ISXf_holdings.csv").dropna()
ISXF.columns = ISXF.iloc[0]
ISXF = ISXF.iloc[1: , :]
ISXF_so = 1375105

ERNS = pd.read_csv("ERNS_holdings.csv").dropna()
ERNS.columns = ERNS.iloc[0]
ERNS = ERNS.iloc[1: , :]
ERNS_so = 6908124

LQDA = pd.read_csv("LQDA_holdings.csv").dropna()
LQDA.columns = LQDA.iloc[0]
LQDA = LQDA.iloc[1: , :]
LQDA_so = 214978445

LQDE = pd.read_csv("LQDA_holdings.csv").dropna()
LQDE.columns = LQDE.iloc[0]
LQDE = LQDE.iloc[1: , :]
LQDE_so = 31083977

etf_shares = [IS15_so, ISXF_so, ERNS_so, LQDA_so, LQDE_so]

## Defining functions for application in this task

In [435]:
def convert(pd_series):
    """
    Takes an input pandas series with it's values as strings and first
    removes any commas, then converts the values into floats
    """
    pd_series = pd_series.str.replace(',','')
    pd_series = pd_series.astype('float')
    return pd_series

def conv_df(data):
    """
    Takes an input pandas dataframe from iShares and converts the Market Value,
    Notional Value, Price, and Weight into floats. 
    The function prints an error statement if the values are not strings; it can 
    only be run on the dataframes once. 
    """

    try:
        # cleaning the data and converting the relevant columns to floats
        data["Market Value"] = convert(data["Market Value"])
        data["Notional Value"] = convert(data["Notional Value"])
        data["Price"] = convert(data["Price"])
        data["Weight (%)"] = convert(data["Weight (%)"])
        data["ISIN"] = data["ISIN"].astype("string")
        
    except:
        print("The values are not strings. You may have run the same script twice.")

    return data


def increment_mvs(data, isins, mean, vol):
    """
    This function takes in an input dataframe from the iShares website.
    Creates a new column called "New Market Value".
    Generates random returns from a normal distribution.
    Then adjusts each previous market value, indexed by the ISIN, to a new market value based
    on the return associated with that ISIN.
    It returns the same dataframe with the New Market Value column.
    """
    
    data["New Market Value"] = ""
    returns = 1 + np.random.normal(mean, vol, len(isins))
    count = 0
    
    for i in range(0, len(isins)):
        if isins[i] in data["ISIN"].values == True:
            data.loc[data["ISIN"] == isins[i], "New Market Value"] = round(float(data.loc[data["Market Value"][data["ISIN"] == isins[i]].index]["Market Value"].values * returns[i]),2)
            count = count + 1 
        if count == len(data):
            break 
    
    return data  

def calc_nav(df, shares, new=False):
    """
    This function calculates the net asset value of the input ETF
    It takes in an ETF dataframe from the iShares website along with the number of outstanding shares
    The function calculates the net asset value by simply summating the market values of each asset - the market 
    values of each asset within the portfolio are already provided so there is no need to multiply anything
    by the relevant weights. 
    The function also takes in a boolean "new" - if new is True, then the new market values are calculated using
    the "New Market Value" column instead of the original "Market Value" column
    """
    if new == False:
        nav = df["Market Value"].sum()/shares
        return round(float(nav), 2)
    
    else:
        nav = (df["New Market Value"][df["ISIN"] != "-"].sum() + df["Market Value"][df["ISIN"] == "-"].sum())/shares
        return round(float(nav), 2)
    
def re_calc_navs(etfs, etf_shares, mean, vol):
    """
    This function takes in a list of dataframes containing the iShares ETF datasets, the number of outstanding shares per 
    dataframe, and also the mean and volatility parameters for a normal distribution from which random returns will be 
    generated to perturb the market values of the contents of the ETFs. 
    The function works by concatenating all the ISINs from each of the dataframes together, only taking the unique ones, 
    then adding a random perturbation to each market value indexed by each ISIN ID in each of the ETF dataframes. 
    The new market values are then used to recalculate the NAV. 
    The entire function is timed with the time outputted. 
    """
    navs = []

    start = timer()

    # concatenating the ISINs from each dataframe together into a numpy array of unique ISINs
    isins = np.unique(np.concatenate([etfs[i]["ISIN"][etfs[i]["ISIN"] != '-'].unique() for i in range(len(etfs))]))

    # adjusting the market values
    for i in range(len(etfs)):
        etfs[i] = increment_mvs(etfs[i], isins, mean, vol)
        navs.append(calc_nav(etfs[i], etf_shares[i], True))

    end = timer()
    
    return navs, timedelta(seconds=end-start)

## Converting each dataframe's columns into the proper Pythonic type

In [416]:
# Converting the column values to their relevant types
IS15 = conv_df(IS15)
ISXF = conv_df(ISXF)
ERNS = conv_df(ERNS)
LQDA = conv_df(LQDA)
LQDE = conv_df(LQDE)

## Calculating the NAV of each ETF statically

In [421]:
# defining list of etfs and list of shares outstanding per etf
etfs = [IS15, ISXF, ERNS, LQDA, LQDE]
etf_shares = [IS15_so, ISXF_so, ERNS_so, LQDA_so, LQDE_so]

for i in range(len(etfs)):
    # Calculate NAV
    start = timer()
    nav = calc_nav(etfs[i], etf_shares[i])
    end = timer()

    og_time = timedelta(seconds=end-start)
    print("NAV: ", nav, "", "Time to calculate: ", og_time, sep="\n")

NAV: 
106.91

Time to calculate: 
0:00:00.000138
NAV: 
135.28

Time to calculate: 
0:00:00.000131
NAV: 
100.41

Time to calculate: 
0:00:00.000202
NAV: 
27.85

Time to calculate: 
0:00:00.000234
NAV: 
192.61

Time to calculate: 
0:00:00.000136


## Adding random normally distributed perturbations to each unique ISIN and re-calculating the NAV of each ETF

In [434]:
# list of dataframes containing each iShares ETF dataset
etfs = [IS15, ISXF, ERNS, LQDA, LQDE]
etf_shares = [IS15_so, ISXF_so, ERNS_so, LQDA_so, LQDE_so]

# normal distribution parameters for perturbing the the market values
mean = 0
vol = 0.01

for i in range(len(etfs)):
    e = etfs[0:i+1]
    navs, time = re_calc_navs(e, etf_shares, mean, vol)
    print("NAV values:", navs)
    print(time)
    print("Increase in time for", len(e), "ETFs:", time/og_time)

NAV values: [106.92]
0:00:00.397289
Increase in time for 1 ETFs: 2921.2426470588234
NAV values: [106.97, 135.19]
0:00:00.993730
Increase in time for 2 ETFs: 7306.838235294118
NAV values: [106.87, 135.22, 100.45]
0:00:01.232013
Increase in time for 3 ETFs: 9058.91911764706
NAV values: [106.87, 135.2, 100.33, 27.84]
0:00:05.961844
Increase in time for 4 ETFs: 43837.08823529412
NAV values: [107.02, 135.22, 100.45, 27.86, 192.58]
0:00:10.160879
Increase in time for 5 ETFs: 74712.3455882353
